In [1]:
import pandas as pd
import graphlab as gb
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
get_ipython().magic(u'matplotlib inline')
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from time import time
from utility import *

A newer version of GraphLab Create (v2.1) is available! Your current version is v1.9.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


This non-commercial license of GraphLab Create for academic use is assigned to freeze.francis@iiitb.org and will expire on July 06, 2017.


2016-11-16 15:38:36,172 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1479290912.log


In [6]:
df=pd.read_csv('loan_cleaned.csv')

In [7]:
#replace emp_length(str) with emp_length(int)
df.replace('n/a', np.nan,inplace=True)
df.emp_length.fillna(value=0,inplace=True)
df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df['emp_length'] = df['emp_length'].astype(int)

In [8]:
#sorted(df.columns)

df.to_csv("loan_cleaned.csv",sep=",",index=False)

In [9]:
numeric_cols=df.describe().columns

In [10]:
sorted(numeric_cols)

['acc_now_delinq',
 'annual_inc',
 'collection_recovery_fee',
 'collections_12_mths_ex_med',
 'delinq_2yrs',
 'dti',
 'emp_length',
 'funded_amnt',
 'funded_amnt_inv',
 'id',
 'inq_last_6mths',
 'installment',
 'int_rate',
 'labels',
 'last_pymnt_amnt',
 'loan_amnt',
 'member_id',
 'mths_since_last_delinq',
 'open_acc',
 'out_prncp',
 'out_prncp_inv',
 'policy_code',
 'pub_rec',
 'recoveries',
 'revol_bal',
 'revol_util',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_acc',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_int',
 'total_rec_late_fee',
 'total_rec_prncp',
 'total_rev_hi_lim']

In [11]:
#df.info()

In [12]:
for col in numeric_cols:
    print col,df[col].unique()
    print ""

id [ 1077501  1077430  1077175 ..., 36241316 36421485 36260758]

member_id [ 1296599  1314167  1313524 ..., 38952731 39142898 38972123]

loan_amnt [  5000.   2500.   2400. ...,  34175.  29525.  29150.]

funded_amnt [  5000.   2500.   2400. ...,  34175.  29525.  29150.]

funded_amnt_inv [  4975.   2500.   2400. ...,  30425.  31350.  30525.]

int_rate [ 10.65  15.27  15.96  13.49   7.9   18.64  21.28  12.69  14.65   9.91
  16.29   6.03  11.71  12.42  14.27  16.77   7.51   8.9   18.25   6.62
  19.91  17.27  17.58  21.67  19.42  20.89  20.3   23.91  19.03  23.13
  22.74  22.35  22.06  24.11   6.    23.52  22.11   7.49  11.99   5.99
  10.99   9.99  18.79  11.49   8.49  15.99  16.49   6.99  12.99  15.23
  14.79   5.42  10.59  17.49  15.62  19.29  13.99  18.39  16.89  17.99
  20.99  22.85  19.69  20.62  20.25  21.36  23.22  21.74  22.48  23.59
  12.62  18.07  11.63   7.91   7.42  11.14  20.2   12.12  19.39  16.11
  17.54  22.64  13.84  16.59  17.19  12.87  20.69   9.67  21.82  19.79
  18.49  

In [13]:
to_remove=['id','member_id','policy_code']

In [14]:
df_num=df[numeric_cols]
df_num.drop(to_remove,axis=1,inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [15]:
df_num.columns

Index([u'loan_amnt', u'funded_amnt', u'funded_amnt_inv', u'int_rate',
       u'installment', u'emp_length', u'annual_inc', u'dti', u'delinq_2yrs',
       u'inq_last_6mths', u'mths_since_last_delinq', u'open_acc', u'pub_rec',
       u'revol_bal', u'revol_util', u'total_acc', u'out_prncp',
       u'out_prncp_inv', u'total_pymnt', u'total_pymnt_inv',
       u'total_rec_prncp', u'total_rec_int', u'total_rec_late_fee',
       u'recoveries', u'collection_recovery_fee', u'last_pymnt_amnt',
       u'collections_12_mths_ex_med', u'acc_now_delinq', u'tot_coll_amt',
       u'tot_cur_bal', u'total_rev_hi_lim', u'labels'],
      dtype='object')

In [16]:
feature_cols=list(df_num.columns)
feature_cols.remove('labels')

In [17]:
labels=df['labels']

In [18]:
feature_cols=['emp_length','loan_amnt','int_rate','annual_inc','dti']

In [19]:
features=df_num[feature_cols]

In [20]:
features = scale(features)

In [21]:
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=300)))

In [22]:
#bench_k_means(KMeans(init='k-means++', n_clusters=7, n_init=10),name="k-means++", data=features)

#bench_k_means(KMeans(init='random', n_clusters=7, n_init=10),name="random", data=features)

In [23]:
reduced_features = PCA(n_components=5).fit_transform(features)
kmeans = KMeans(init='k-means++', n_clusters=7, n_init=10)
kmeans.fit(reduced_features)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_features[:, 0].min() - 1, reduced_features[:, 0].max() + 1
y_min, y_max = reduced_features[:, 1].min() - 1, reduced_features[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_features[:, 0], reduced_features[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

ValueError: Incorrect number of features. Got 2 features, expected 5

In [ ]:
clusterer=KMeans(n_clusters=10,init='k-means++')
clusterer.fit(features)
cluster_ids=clusterer.labels_

In [ ]:
df_num['cluster_id']=cluster_ids

In [ ]:
l=sorted(df_num['cluster_id'].unique())

In [ ]:
for i in l:
    tmp=df_num[df_num['cluster_id']==i]
    print "cluster_id:"+str(i),"#good loan:"+str(sum(tmp['labels']==1)),"#bad loan:"+str(sum(tmp['labels']==0))